# Module 12.2: Billing Integration

**Duration:** 42 minutes  
**Prerequisites:** M12.1 (Usage Metering), Level 2 complete

---

## Section 1: Introduction & Hook

### The Problem

You built usage metering in M12.1. You track every query, embedding, and token consumed. Your ClickHouse dashboard shows exactly what each tenant is using.

**But:** You're still manually invoicing customers at the end of each month. You spend 8-10 hours exporting usage data, calculating costs, creating invoices, and chasing payments.

- At 10 customers: Manageable
- At 50 customers: 2 full days every month
- At 100+ customers: Unsustainable

### The Solution

Integrate Stripe to automate your entire billing lifecycle:
- Subscription management
- Usage-based charges
- Payment retries with dunning logic
- Automated invoice generation

### Learning Objectives

By the end of this module, you'll:
- Integrate Stripe for subscription and usage-based billing
- Automate invoice generation from M12.1 metering data
- Handle payment failures with proper dunning logic
- Manage subscription lifecycles (trials, upgrades, cancellations)
- **Understand when manual billing is acceptable and what alternatives exist**

## Learning Arc

**Purpose**: Master automated subscription and usage-based billing by integrating Stripe into a multi-tenant SaaS platform, transforming M12.1 usage data into revenue collection.

**Concepts Covered**:
- Customer lifecycle management (idempotent sync)
- Two-tier pricing models (base fee + usage overages)
- Payment failure handling with escalating dunning logic
- Webhook event processing for real-time billing updates
- Production failure modes and prevention strategies

**After Completing**: You'll automate billing for 10-500 customers, eliminating 8-10 hours/month of manual invoicing, with confidence in handling payment failures, subscription changes, and edge cases.

**Context in Track L3.M12**: This builds on M12.1 (Usage Metering) and prepares you for M12.3 (Self-Service Onboarding), completing the monetization stack for multi-tenant RAG systems.

In [ ]:
# Setup and imports
import os
import sys
from datetime import datetime, timedelta
import json

# OFFLINE mode for L3 consistency
OFFLINE = os.getenv("OFFLINE", "false").lower() == "true"
if OFFLINE:
    print("⚠️ Running in OFFLINE mode — Stripe/ClickHouse calls will be skipped (mocked).")

# Import our module
from src.l3_m12_billing_integration import (
    StripeBillingManager,
    UsageSyncService,
    DunningManager
)
from config import Config

print("✓ Imports successful")
print(f"Stripe configured: {Config.is_stripe_configured()}")

# Expected: ✓ Imports successful / Stripe configured: True/False

## Section 2: Prerequisites & Setup

### Starting Point Verification

Your M12.1 system should have:
- ✅ Usage metering capturing all consumption events
- ✅ ClickHouse aggregating usage by tenant and time period
- ✅ Dashboard showing real-time usage statistics
- ✅ API endpoint: `GET /api/tenants/{tenant_id}/usage?start_date=X&end_date=Y`

**The gap we're filling:** You have usage data but no automated payment collection.

### Dependencies & Setup

**Cost Awareness:** Stripe charges **2.9% + $0.30** per successful transaction.
Budget $150-500/month for 50-100 tenants.

In [ ]:
# Verify configuration
Config.print_config_status()

# Load example data
with open('example_data.json', 'r') as f:
    example_data = json.load(f)

print(f"\n✓ Loaded {len(example_data['customers'])} example customers")
print(f"✓ Loaded {len(example_data['pricing_plans'])} pricing plans")

# Expected: Configuration status printed / Example data loaded

## Section 3: Theory Foundation - Stripe Billing Concepts

### The Stripe Data Model

Think of Stripe like a parallel database for billing:

- **Customers:** Mirror your tenants in Stripe
- **Subscriptions:** Recurring billing plans (monthly/annual)
- **Products & Prices:** What you're selling (e.g., 'Pro Plan - $99/mo')
- **Usage Records:** Consumption data you send (queries, tokens)
- **Invoices:** Generated automatically based on subscription + usage
- **Payment Methods:** Credit cards stored securely in Stripe

### How It Works (5 Steps)

1. **Customer Creation:** When tenant signs up, create Stripe Customer object
2. **Subscription Creation:** Attach subscription defining base price + usage metering
3. **Usage Reporting:** Throughout month, send usage records to Stripe
4. **Invoice Generation:** At billing period end, Stripe auto-generates invoice
5. **Payment Handling:** If payment succeeds → done. If fails → retry with dunning

### Why This Matters for Production

- **Scalability:** Handles millions of transactions without custom code
- **Compliance:** PCI-compliant payment handling (you never touch credit cards)
- **Automation:** Invoicing, retries, receipts all handled automatically
- **Reporting:** Built-in revenue analytics and tax calculation

**Common Misconception:** "I can build this myself in a weekend." 
**Reality:** Payment processing has edge cases (partial refunds, prorations, failed retries, dispute handling) that take months to build properly.

## Section 4: Hands-On Implementation

### Step 1: Customer Management

Sync your tenants to Stripe customers with idempotent creation.

In [ ]:
# Step 1: Create a customer
billing = StripeBillingManager()

# Get example customer from data
customer_data = example_data['customers'][0]
print(f"Creating customer for: {customer_data['name']}")

if Config.is_stripe_configured():
    customer_id = billing.create_customer(
        tenant_id=customer_data['tenant_id'],
        email=customer_data['email'],
        name=customer_data['name']
    )
    print(f"✓ Customer ID: {customer_id}")
else:
    print("⚠️ Skipping API call (no Stripe key configured)")
    customer_id = "cus_example123"

# Expected: Customer created / or skipped if no key

### Step 2: Subscription Management

Create subscriptions with two-tier pricing: base monthly fee + usage overages.

**Pricing Plans:**
- **Starter**: $29/mo + 10K included queries
- **Pro**: $99/mo + 100K included queries  
- **Enterprise**: $499/mo + 1M included queries

In [ ]:
# Step 2: Create subscription
if Config.is_stripe_configured() and customer_id:
    subscription = billing.create_subscription(
        customer_id=customer_id,
        plan_type=customer_data['plan_type'],
        tenant_id=customer_data['tenant_id'],
        trial_days=14
    )
    if subscription:
        print(f"✓ Subscription: {subscription['subscription_id']}")
        print(f"  Status: {subscription['status']}")
        print(f"  Trial ends: {subscription.get('trial_end', 'N/A')}")
else:
    print("⚠️ Skipping subscription creation")
    subscription = {"subscription_id": "sub_example456", "status": "trialing"}

# Expected: Subscription created with 14-day trial / or skipped

### Step 3: Usage Reporting

Report daily usage from M12.1 to Stripe for accurate billing.

In [ ]:
# Step 3: Report usage
sync_service = UsageSyncService(billing)

# Use example usage data
usage_sync_data = example_data['usage_data']
print(f"Syncing usage for {len(usage_sync_data)} tenants...")

if Config.is_stripe_configured():
    results = sync_service.sync_daily_usage(usage_sync_data)
    success_count = sum(1 for r in results if r.get('success'))
    print(f"✓ Synced: {success_count}/{len(results)}")
else:
    print("⚠️ Skipping usage sync (no Stripe key)")

# Expected: Usage synced for all tenants / or skipped

### Step 4: Dunning Logic for Payment Failures

Implement escalating retry strategy:
- **Day 1**: Soft reminder email
- **Day 4**: Warning email
- **Day 7**: Reduce rate limits (soft suspension)
- **Day 8+**: Full service suspension

In [ ]:
# Step 4: Test dunning logic
dunning = DunningManager()

# Simulate payment failures at different stages
for scenario in example_data['dunning_scenarios']:
    result = dunning.process_failed_payment(
        tenant_id=scenario['tenant_id'],
        failure_count=scenario['failure_count'],
        invoice_amount=99.00
    )
    print(f"Failure {scenario['failure_count']}: {result['action']} - {scenario['description']}")

# Test reactivation
reactivation = dunning.reactivate_tenant('tenant_002')
print(f"\n✓ Reactivation: {reactivation['action']}")

# Expected: Escalating dunning actions shown / reactivation successful

## Section 5: Reality Check - What This DOESN'T Do

### Limitations You Need to Know

1. **Complex Proration Scenarios**
   - Mid-month upgrades/downgrades get messy
   - Requires manual intervention for edge cases
   - Workaround: Review prorations manually first few months

2. **Billing Disputes**
   - Customers will still dispute charges if confused about usage-based pricing
   - Need clear communication: "You used 125K queries (25K over limit = $20 overage)"
   - **Impact**: Expect 2-5% of invoices to be disputed in first 6 months

3. **Multi-Currency and Tax Complexity**
   - Stripe automatic tax works for US/EU, but some regions require manual config
   - Currency conversion happens at payment time, not usage reporting time
   - When you'll hit this: First international customer (months 3-6)

### Trade-offs You Accepted

- **Complexity**: Added 500+ lines of code (Stripe SDK, webhooks, dunning)
- **Performance**: Webhook endpoints must respond <5 seconds or Stripe retries
- **Cost**: Stripe fees (2.9% + $0.30) eat into margins
  - At 100 customers × $99/month = $9,900 revenue
  - Stripe fees: ~$315/month (3.2% of revenue)
- **Vendor Lock-in**: Migrating off Stripe requires rebuilding entire billing system

### When This Approach Breaks

**At scale >1000 paying customers:**
- Webhook volume becomes bottleneck (>10K events/day)
- Need dedicated webhook processing service (Celery queue, not synchronous)
- Stripe fee percentage negotiable at this scale

## Section 6: Common Failures (Production Issues)

### Failure 1: Webhook Missed Events (Payment Not Recorded)

**Root Cause**: Webhook endpoint takes >5 seconds, Stripe times out and stops retrying after 3 days.

**What You'll See**:
- Customer paid but database shows `billing_status='payment_failed'`
- Customer complaint: "I paid but my service is suspended"

**The Fix**:
```python
# BEFORE (slow, synchronous):
@app.post("/webhooks/stripe")
async def webhook(request):
    event = verify_webhook(request)
    await process_event(event)  # Slow!
    return {"status": "success"}

# AFTER (fast, asynchronous):
@app.post("/webhooks/stripe")
async def webhook(request):
    event = verify_webhook(request)
    background_tasks.add_task(process_event, event)
    return {"status": "success"}  # Returns in <100ms
```

**Prevention**:
- Webhook endpoints should ONLY verify signature and queue the event
- All processing happens asynchronously (Celery, SQS, Pub/Sub)
- Monitor webhook response time (<500ms target)

---

### Failure 2: Billing Calculation Errors (Over/Undercharging)

**Root Cause**: Usage sync reports wrong quantity (e.g., counting cached + database queries = double counting).

**The Fix**:
- Single source of truth for usage (ClickHouse events table)
- Use `action=\"set\"` not `\"increment\"` for idempotency
- Automated daily comparison: Stripe usage records vs ClickHouse aggregations

---

### Failure 3: Invoice Generation Failures (Customer Not Billed)

**Root Cause**: Subscription has no payment method attached. Trial ends, Stripe can't charge, invoice stays in \"draft\" forever.

**The Fix**:
- Require payment method at signup (before trial starts)
- Send reminders 3 days before trial ends
- Monitor subscriptions in \"incomplete\" status daily

---

### Failure 4: Payment Retry Bugs (Premature Suspension)

**Root Cause**: Webhooks arrive out of order. `payment_failed` webhook processed after `payment_succeeded` already happened.

**The Fix**:
```python
async def handle_payment_failed(invoice_data):
    # Always check CURRENT state in Stripe
    latest_invoice = stripe.Invoice.retrieve(invoice_id)
    
    # Only suspend if STILL unpaid
    if latest_invoice.status in [\"open\", \"uncollectible\"]:
        if latest_invoice.attempt_count >= 3:
            await suspend_tenant(tenant_id)
    else:
        logger.info(f\"Ignoring old webhook - invoice already paid\")
```

---

### Failure 5: State Conflicts (Stripe vs Database Mismatch)

**Root Cause**: Customer cancels in Stripe Dashboard, your database still shows `billing_status='active'`.

**The Fix**:
- Daily reconciliation job (Stripe → DB sync)
- Webhook reliability monitoring
- Admin dashboard showing discrepancies

## Section 7: DECISION CARD

### Automated Stripe Billing - Decision Card

---

**✅ BENEFIT: Automated Revenue Collection & Scaling**

Eliminates manual invoicing (saves 8-10 hours/month at 10+ customers), automatically retries failed payments with dunning logic, scales to 500+ customers without additional work. Typical ROI: 40-hour setup cost paid back in 4-6 months via time savings.

---

**❌ LIMITATION: Complex Edge Cases Require Manual Intervention**

Prorations on mid-month plan changes are messy, international tax compliance needs manual configuration for some regions, billing disputes from confused customers require support team involvement. **Expect 2-5% of invoices to need manual review in first 6 months.**

---

**💰 COST: Stripe Fees + Infrastructure + Time Investment**

- Stripe fees: 2.9% + $0.30 per transaction (~3-4% of revenue)
- Infrastructure: $100-200/month (Celery workers, monitoring)
- Engineering: 40 hours setup + 5 hours/month maintenance
- **Total: 4-5% of revenue at 100 customers scale**

---

**🤔 USE WHEN: 10+ Customers with Monthly/Annual Subscriptions**

Use if you have (or will reach within 6 months):
- >10 paying customers
- Monthly or annual billing cycles
- Validated pricing model (not changing every 2-3 months)
- SMB/mid-market customers who accept credit card payments

---

**🚫 AVOID WHEN: <10 Customers or Enterprise-Only Sales**

Skip if you have:
- <10 customers and no growth plan (manual invoicing faster at this scale)
- All customers are enterprise with annual contracts and net-30 terms
- Pricing model still experimental (changing monthly)

---

### Alternative Solutions

| Your Situation | Best Choice | Why |
|----------------|-------------|-----|
| <10 customers, B2B only | Manual invoicing (QuickBooks) | Time efficient at small scale |
| 10-500 customers, SMB/B2C | **Stripe** (today's approach) | Best balance of features/cost |
| >500 customers, complex pricing | Chargebee/Recurly | Handles complexity |
| International heavy (>50%) | PayPal/Braintree | Better global coverage |
| >$1M ARR, custom needs | Kill Bill (open source) | Ownership and control |

---

### When NOT to Use

**Scenario 1**: Very Small Customer Base (<10 customers paying >$500/mo)
- **Why it fails**: 40-hour setup vs 10 hours/month manual work = 4-6 month payback
- **Use instead**: Manual invoicing (QuickBooks/Xero)

**Scenario 2**: Enterprise-Only with Custom Contracts
- **Why it fails**: Enterprise expects invoices, not credit card charges. Payment terms net-30/60/90.
- **Use instead**: Manual invoicing with accounting software

**Scenario 3**: Uncertain Pricing Model (changing every 2-3 months)
- **Why it fails**: Stripe price IDs are immutable, migration between models is complex
- **Use instead**: Wait until pricing stabilizes

## Section 8: Summary & Next Steps

### What You Accomplished

✅ Integrated Stripe for subscription and usage-based billing  
✅ Automated invoice generation from M12.1 usage data  
✅ Implemented payment retry logic with dunning  
✅ Handled subscription lifecycle events

### Key Takeaways

1. **Manual billing is acceptable for <10 customers** - Don't over-engineer
2. **Always verify webhook signatures** - Security critical
3. **Expect 2-5% of invoices to need manual review** - Complex edge cases exist
4. **Daily reconciliation prevents state mismatches** - Stripe vs your database

### Production Checklist

- [ ] Stripe live API keys in production environment
- [ ] Webhook endpoint configured in Stripe Dashboard
- [ ] Webhook signature verification enabled
- [ ] Celery workers for async webhook processing
- [ ] Daily usage sync cron job scheduled
- [ ] Monitoring and alerts configured
- [ ] Test payment failure scenarios

### Next Module

**M12.3: Self-Service Tenant Onboarding**  
Automate signup and provisioning so customers can onboard themselves without your involvement.